## Data Acquisition

Used Kaggle to obtain data on New York City Weather and New York City collisions. We will be combining these two datasets to comapre collisions with weather.

Weather dataset: https://www.kaggle.com/datasets/aadimator/nyc-weather-2016-to-2022

Collisions dataset: https://www.kaggle.com/datasets/amanaayush/nyc-traffic-accidents

In [1]:
import pandas as pd
import datetime

In [3]:
collisions = pd.read_csv("NYC_Collisions.csv.zip")
weather = pd.read_csv("NYC_Weather_2016_2022.csv.zip")
print(f"""
    Collisions DataFrame shape: {collisions.shape}
    Weather DataFrame shape: {weather.shape}
      """)


    Collisions DataFrame shape: (238421, 18)
    Weather DataFrame shape: (59760, 10)
      


In [4]:
collisions.head()

,Collision ID,Date,Time,Borough,Street Name,Cross Street,Latitude,Longitude,Contributing Factor,Vehicle Type,Persons Injured,Persons Killed,Pedestrians Injured,Pedestrians Killed,Cyclists Injured,Cyclists Killed,Motorists Injured,Motorists Killed
0,4491746,2021-01-01,20:00:00,Bronx,Bruckner Expressway,NaN,40.83398,-73.826350,Pavement Slippery,Passenger Vehicle,0.0,0,0,0,0,0,0,0
1,4441905,2021-01-01,05:28:00,Brooklyn,Lafayette Avenue,NaN,40.68730,-73.973656,Unspecified,Passenger Vehicle,0.0,0,0,0,0,0,0,0
2,4382769,2021-01-01,06:00:00,Staten Island,West Shore Expressway,NaN,NaN,NaN,Fell Asleep,Passenger Vehicle,0.0,0,0,0,0,0,0,0
3,4380949,2021-01-01,19:30:00,Bronx,Sedgwick Avenue,Vancortlandt Avenue West,40.88270,-73.892730,NaN,Not Reported,0.0,0,0,0,0,0,0,0
4,4380940,2021-01-01,07:40:00,Brooklyn,Cortelyou Road,Mc Donald Avenue,40.63791,-73.978640,Unspecified,Passenger Vehicle,0.0,0,0,0,0,0,0,0


In [5]:
weather.head()

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2016-01-01T00:00,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0
1,2016-01-01T01:00,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0
2,2016-01-01T02:00,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0
3,2016-01-01T03:00,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0
4,2016-01-01T04:00,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0


In [6]:
collisions.dtypes

,0
Collision ID,int64
Date,object
Time,object
Borough,object
Street Name,object
Cross Street,object
Latitude,float64
Longitude,float64
Contributing Factor,object
Vehicle Type,object


In [7]:
weather.dtypes

,0
time,object
temperature_2m (°C),float64
precipitation (mm),float64
rain (mm),float64
cloudcover (%),float64
cloudcover_low (%),float64
cloudcover_mid (%),float64
cloudcover_high (%),float64
windspeed_10m (km/h),float64
winddirection_10m (°),float64


Need to merge the two databases based on date. The `time` value in the `weather` database needs to be converted to date time and then seprated out the date and time. There is already a `Date` and `Time` column in the `collisions` database, so we can merge based on dates.

In [8]:
#Add a new date column based on time column in datetime format
weather['date'] = pd.to_datetime(weather['time'])
collisions['Date'] = pd.to_datetime(collisions['Date'])

In [9]:
#Extract the date and time from the date column
weather['time'] = weather['date'].dt.time
weather['date'] = pd.to_datetime(weather['date'].dt.date)
weather.dtypes

,0
time,object
temperature_2m (°C),float64
precipitation (mm),float64
rain (mm),float64
cloudcover (%),float64
cloudcover_low (%),float64
cloudcover_mid (%),float64
cloudcover_high (%),float64
windspeed_10m (km/h),float64
winddirection_10m (°),float64


We might want to look at collisions in terms of time, but merging on date and time would mean many rows. We can separate out the times based on time of day (i.e. morning, midday, and night). This new column called `time_of_day` will be a categorial variable. This column will be morning for 2am to 10am, midday for 10am to 6pm, and night for 6pm to 2am.

In [10]:
# Define a function to categorize time_of_day
def categorize_time_of_day(time):
    # Extract hour from the time object
    hour = int(str(time)[:2])
    if 2 <= hour < 10:
        return 'morning'
    elif 10 <= hour < 18:
        return 'midday'
    else:
        return 'night'

In [11]:
# Apply the function to the 'time' column to create the 'time_of_day' column
weather['time_of_day'] = weather['time'].apply(categorize_time_of_day)
collisions['time_of_day'] = collisions['Time'].apply(categorize_time_of_day)

### Merge data

In [12]:
print(
    f"""Most recent date in collisions data: {collisions.sort_values(by='Date', ascending=False)['Date'].iloc[0].date()}
Oldest date in collisions data: {collisions.sort_values(by='Date')['Date'].iloc[0].date()}
""")

Most recent date in collisions data: 2023-04-09
Oldest date in collisions data: 2021-01-01



In [13]:
print(
    f"""Most recent date in weather data: {weather.sort_values(by='date', ascending=False)['date'].iloc[0].date()}
Oldest date in weather data: {weather.sort_values(by='date')['date'].iloc[0].date()}
""")

Most recent date in weather data: 2022-10-25
Oldest date in weather data: 2016-01-01



Based on the oldest and most recent dates in both databases, we can use dates from 1/1/2021 to 10/25/2022. Weather is taken every hour during the day.

In [14]:
weather['date'].value_counts().sort_index()

,count
date,
2016-01-01,24
2016-01-02,24
2016-01-03,24
2016-01-04,24
2016-01-05,24
...,...
2022-10-21,24
2022-10-22,24
2022-10-23,24


In [15]:
weather.columns

Index(['time', 'temperature_2m (°C)', 'precipitation (mm)', 'rain (mm)',
       'cloudcover (%)', 'cloudcover_low (%)', 'cloudcover_mid (%)',
       'cloudcover_high (%)', 'windspeed_10m (km/h)', 'winddirection_10m (°)',
       'date', 'time_of_day'],
      dtype='object')

In [16]:
#Create an aggergated weather dataframe with the average values for each day separted by time of day
weather_avg = weather.groupby(['date', 'time_of_day']).agg({
    'temperature_2m (°C)':'mean',
    'precipitation (mm)':'mean',
    'rain (mm)':'mean',
    'cloudcover (%)':'mean',
    'cloudcover_low (%)':'mean',
    'cloudcover_mid (%)':'mean',
    'cloudcover_high (%)':'mean',
    'windspeed_10m (km/h)':'mean',
    'winddirection_10m (°)':'mean'
}).reset_index()
weather_avg.sort_values(by='date',inplace=True)


In [17]:
#Merge data on date and ttime_of_day
collisions['date'] = collisions['Date']

merged_data = pd.merge(collisions, weather_avg, on=['date','time_of_day'], how='inner').sort_values(by='date').reset_index(drop=True)

In [18]:
merged_data.columns

Index(['Collision ID', 'Date', 'Time', 'Borough', 'Street Name',
       'Cross Street', 'Latitude', 'Longitude', 'Contributing Factor',
       'Vehicle Type', 'Persons Injured', 'Persons Killed',
       'Pedestrians Injured', 'Pedestrians Killed', 'Cyclists Injured',
       'Cyclists Killed', 'Motorists Injured', 'Motorists Killed',
       'time_of_day', 'date', 'temperature_2m (°C)', 'precipitation (mm)',
       'rain (mm)', 'cloudcover (%)', 'cloudcover_low (%)',
       'cloudcover_mid (%)', 'cloudcover_high (%)', 'windspeed_10m (km/h)',
       'winddirection_10m (°)'],
      dtype='object')

In [19]:
#only keep the columns we need
merged_data = merged_data[['date','time_of_day','Collision ID','Borough','Contributing Factor','Persons Injured','Persons Killed','Pedestrians Injured','Pedestrians Killed','Cyclists Injured',
       'Cyclists Killed', 'Motorists Injured', 'Motorists Killed','temperature_2m (°C)','precipitation (mm)','rain (mm)','cloudcover (%)','cloudcover_low (%)','cloudcover_mid (%)','cloudcover_high (%)','windspeed_10m (km/h)','winddirection_10m (°)']]

In [20]:
merged_data.head()

,date,time_of_day,Collision ID,Borough,Contributing Factor,Persons Injured,Persons Killed,Pedestrians Injured,Pedestrians Killed,Cyclists Injured,...,Motorists Killed,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2021-01-01,night,4491746,Bronx,Pavement Slippery,0.0,0,0,0,0,...,0,3.1375,0.125,0.125,94.875,14.250,95.625,99.50,7.9625,145.375
1,2021-01-01,midday,4381244,Manhattan,Unsafe Speed,0.0,0,0,0,0,...,0,-1.3375,0.000,0.000,28.625,0.875,2.750,87.25,6.9375,47.375
2,2021-01-01,midday,4380799,Queens,Driver Inattention/Distraction,1.0,0,0,0,0,...,0,-1.3375,0.000,0.000,28.625,0.875,2.750,87.25,6.9375,47.375
3,2021-01-01,night,4381095,Brooklyn,Unspecified,0.0,0,0,0,0,...,0,3.1375,0.125,0.125,94.875,14.250,95.625,99.50,7.9625,145.375
4,2021-01-01,morning,4380755,Queens,Unspecified,0.0,0,0,0,0,...,0,0.3875,0.000,0.000,21.750,0.000,1.125,70.00,7.9375,339.375


## Data Cleaning and Handling Inconsistencies

In [21]:
#Check if there are incorrect data types
merged_data.dtypes

,0
date,datetime64[ns]
time_of_day,object
Collision ID,int64
Borough,object
Contributing Factor,object
Persons Injured,float64
Persons Killed,int64
Pedestrians Injured,int64
Pedestrians Killed,int64
Cyclists Injured,int64


The data types appear to be well-defined and appropriate for the dataset. The dataset contains:
- 1 column of `datatime64[ns]`
- 10 columns of `int64`
- 8 columns of `float64`
- 3 columns of `object`

In [22]:
#Check the number and percentage of missing values for each column
missing_data = pd.DataFrame({
    'Missing Count': merged_data.isnull().sum(),
    'Missing Percentage': merged_data.isnull().sum() / len(merged_data) * 100
})

missing_data = missing_data.sort_values(by='Missing Percentage', ascending=False).reset_index()
missing_data

,index,Missing Count,Missing Percentage
0,Borough,5897,3.008198
1,winddirection_10m (°),2037,1.039121
2,temperature_2m (°C),2037,1.039121
3,windspeed_10m (km/h),2037,1.039121
4,cloudcover_high (%),2037,1.039121
5,cloudcover_mid (%),2037,1.039121
6,cloudcover_low (%),2037,1.039121
7,cloudcover (%),2037,1.039121
8,rain (mm),2037,1.039121
9,precipitation (mm),2037,1.039121


## Exploratory Data Analysis (EDA)

## Data Preprocessing and Feature Engineering